In [1]:
import os
import json
import numpy as np
import pandas as pd
import SimpleITK as sitk
from tqdm import tqdm

import radiomics
import logging

# Suppress non-critical logs
radiomics.logger.setLevel(logging.ERROR)
logging.getLogger("radiomics").setLevel(logging.ERROR)

from radiomics import featureextractor

# -------- Utilities --------

def normalize_nonzero(image_np):
    nonzero = image_np[image_np != 0]
    if nonzero.size == 0:
        return image_np
    return (image_np - np.mean(nonzero)) / np.std(nonzero)

# -------- Radiomics Setup --------
extractor = featureextractor.RadiomicsFeatureExtractor()
extractor.enableFeatureClassByName('shape')
extractor.settings['force2D'] = False

radiomics_shape_feature_names = [
    "Elongation", "Flatness", "LeastAxisLength", "MajorAxisLength",
    "Maximum2DDiameterColumn", "Maximum2DDiameterRow", "Maximum2DDiameterSlice",
    "Maximum3DDiameter", "MeshVolume", "MinorAxisLength", "Sphericity",
    "SurfaceArea", "SurfaceVolumeRatio", "VoxelVolume"
]

feature_names = [
    "mean_baseline", "peak", "auc", "range", "wash_in", "wash_out",
    "voxel_mean_over_time", "segmentation_failed"
] + radiomics_shape_feature_names

def extract_features_multiphase(image_paths, mask_path):
    mask = sitk.ReadImage(mask_path)
    mask_array = sitk.GetArrayFromImage(mask).astype(bool)
    image_for_shape = sitk.ReadImage(image_paths[1])  # Use post-contrast

    try:
        result = extractor.execute(image_for_shape, mask_path)
        shape_feats = {
            k.split("_")[-1]: v for k, v in result.items()
            if k.startswith("original_shape_")
        }
    except Exception as e:
        print(f"❌ Radiomics failed: {mask_path} | {e}")
        shape_feats = {feat: 0 for feat in radiomics_shape_feature_names}

    if np.sum(mask_array) == 0 or all(v == 0 for v in shape_feats.values()):
        return {
            "mean_baseline": 0, "peak": 0, "auc": 0, "range": 0,
            "wash_in": 0, "wash_out": 0, "voxel_mean_over_time": 0,
            "segmentation_failed": 1,
            **{feat: 0 for feat in radiomics_shape_feature_names}
        }

    dce_volumes = []
    for img_path in image_paths:
        img = sitk.ReadImage(img_path)
        img_array = sitk.GetArrayFromImage(img)
        clipped = np.clip(img_array, np.percentile(img_array, 0.5), np.percentile(img_array, 99.5))
        normed = normalize_nonzero(clipped)
        dce_volumes.append(normed)

    dce_4d = np.stack(dce_volumes, axis=-1)

    intensity_curve = [np.mean(dce_4d[..., t][mask_array]) for t in range(dce_4d.shape[-1])]
    intensity_curve = np.array(intensity_curve)

    baseline = intensity_curve[0]
    peak = np.max(intensity_curve)
    time_to_peak = np.argmax(intensity_curve)
    auc = np.sum(intensity_curve)
    wash_in = (peak - baseline) / (time_to_peak + 1e-6)
    wash_out = (peak - intensity_curve[-1]) / (len(intensity_curve) - time_to_peak + 1e-6)
    intensity_range = peak - np.min(intensity_curve)
    voxel_mean_over_time = dce_4d[mask_array].mean()

    return {
        "mean_baseline": baseline, "peak": peak, "auc": auc, "range": intensity_range,
        "wash_in": wash_in, "wash_out": wash_out, "voxel_mean_over_time": voxel_mean_over_time,
        "segmentation_failed": 0, **shape_feats
    }

# ------------------ Main Script ------------------
def extract_all_features(json_path, output_path):
    with open(json_path, "r") as f:
        folds = json.load(f)

    fold_entries = folds["fold_0"]["train"] + folds["fold_0"]["val"]

    all_feats = []
    failed_ids = []

    for entry in tqdm(fold_entries, desc="Extracting features"):
        pid = os.path.basename(entry["mask"]).split(".")[0]
        try:
            feats = extract_features_multiphase(entry["image"], entry["mask"])
            feats["patient_id"] = pid
            feats["pcr"] = entry["pcr"]
            all_feats.append(feats)
        except Exception as e:
            print(f"❌ Failed {pid}: {e}")
            failed_ids.append(pid)

    print(f"✅ Extracted: {len(all_feats)}")
    print(f"🚫 Failed: {len(failed_ids)}")

    pd.DataFrame(all_feats).to_csv(output_path, index=False)

    if failed_ids:
        with open("failed_patients.txt", "w") as f:
            f.write("\n".join(failed_ids))
            
# -------- Run Once --------
extract_all_features("MAMA-MIA_Challenge/machine_learning/5_folds/5fold_split_stratified_updated.json", "MAMA-MIA_Challenge/machine_learning/5_folds/all_features.csv")

Extracting features: 100%|██████████| 1506/1506 [44:51<00:00,  1.79s/it] 

✅ Extracted: 1506
🚫 Failed: 0
